In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from thesislib.utils import pathutils
import json
import hashlib

In [3]:
from thesislib.utils.ml import process

In [4]:
data_dir = pathutils.get_data_file("05_27_nlice")
nlice_data_dir = os.path.join(data_dir, "nlice")
analysis_data_dir = os.path.join(data_dir, "analysis")

In [10]:
nlice_module_dir = "/Users/teliov/TUD/symcat-to-synthea/output/module_ai_med_adv"
nlice_data_csv = pathutils.get_data_file("05_27_nlice/ai/output_med_ai_adv/symptoms/csv/symptoms100k.csv")

In [5]:
nlice_op_data_dir = os.path.join(nlice_data_dir, "data")
nlice_parsed_data_dir = os.path.join(analysis_data_dir, "parsed")

In [8]:
data_dir = pathutils.get_data_file("05_27_nlice")
basic_data_dir = os.path.join(data_dir, "basic")
basic_symptom_map_file = os.path.join(basic_data_dir, "symptom_db.json")
basic_condition_map_file = os.path.join(basic_data_dir, "condition_db.json")

In [6]:
transformation_map_file = os.path.join(nlice_op_data_dir, "transformation_map.json")
encoding_map_file = os.path.join(nlice_op_data_dir, "encoding_map.json")
reduction_map_file = os.path.join(nlice_op_data_dir, "reduction_map.json")
with open(transformation_map_file) as fp:
    transformation_map = json.load(fp)
with open(encoding_map_file) as fp:
    encoding_map = json.load(fp)
with open(reduction_map_file) as fp:
    reduction_map = json.load(fp)

In [7]:
import pathlib
pathlib.Path(analysis_data_dir).mkdir(exist_ok=True, parents=True)

In [11]:
# parse the train set and let's train
parsed_train = process.parse_data(
    nlice_data_csv,
    basic_condition_map_file,
    basic_symptom_map_file,
    nlice_parsed_data_dir,
    is_nlice=True,
    transform_map=transformation_map,
    encode_map=encoding_map,
    reduce_map=reduction_map
)

In [13]:
import re
import pandas as pd
import numpy as np

In [27]:
regex = re.compile("(\d{1,2})\|(\d{1,3}),")

In [32]:
df = pd.read_csv(parsed_train, index_col="Index")

In [33]:
symptoms = df.SYMPTOMS

In [34]:
def extract_headache(item):
    match = regex.search(item)
    if match is None:
        return -1
    groups = [int(grp) for grp in match.groups()]
    if groups[0] != 9:
        return -1
    return groups[1]

In [35]:
headache = symptoms.apply(extract_headache)

In [38]:
headache_only = headache.loc[headache >= 0]

In [46]:
headache_only.groupby(headache_only).count()

SYMPTOMS
1       39
22     160
23      43
24    7677
26    1019
27     419
28    3451
30    2390
31     933
32    8202
38      32
39       6
40      78
42     254
43      87
44     648
46     656
47     185
48    5285
Name: SYMPTOMS, dtype: int64

In [50]:
encoding_map['headache']

{'0-None;0-None;0-None;0-None': 1,
 '0-None;0-None;long;0-None': 2,
 '0-None;0-None;medium;0-None': 3,
 '0-None;0-None;short;0-None': 4,
 '0-None;mild;0-None;0-None': 5,
 '0-None;mild;long;0-None': 6,
 '0-None;mild;medium;0-None': 7,
 '0-None;mild;short;0-None': 8,
 '0-None;moderate;0-None;0-None': 9,
 '0-None;moderate;long;0-None': 10,
 '0-None;moderate;medium;0-None': 11,
 '0-None;moderate;short;0-None': 12,
 '0-None;severe;0-None;0-None': 13,
 '0-None;severe;long;0-None': 14,
 '0-None;severe;medium;0-None': 15,
 '0-None;severe;short;0-None': 16,
 'aching;0-None;0-None;0-None': 17,
 'aching;0-None;long;0-None': 18,
 'aching;0-None;medium;0-None': 19,
 'aching;0-None;short;0-None': 20,
 'aching;mild;0-None;0-None': 21,
 'aching;mild;long;0-None': 22,
 'aching;mild;medium;0-None': 23,
 'aching;mild;short;0-None': 24,
 'aching;moderate;0-None;0-None': 25,
 'aching;moderate;long;0-None': 26,
 'aching;moderate;medium;0-None': 27,
 'aching;moderate;short;0-None': 28,
 'aching;severe;0-None